#DistilBert on GoEmotions dataset

#### Download Dataset and label texts


In [1]:
!wget https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv
!wget https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/dev.tsv
!wget https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv
!wget https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt

--2021-04-22 02:25:58--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3519053 (3.4M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   3.36M  13.5MB/s    in 0.2s    

2021-04-22 02:25:58 (13.5 MB/s) - ‘train.tsv’ saved [3519053/3519053]

--2021-04-22 02:25:59--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 O

#### Load dataset from tsv

In [9]:
import pandas as pd
import numpy as np

In [10]:
df_train = pd.read_csv("/content/train.tsv", sep='\t', header =None)
df_test = pd.read_csv("/content/test.tsv", sep='\t', header =None)
df_dev = pd.read_csv("/content/dev.tsv", sep='\t', header =None)

In [11]:
df_dev.head()

,0,1,2
0,Is this in New Orleans?? I really feel like th...,27,edgurhb
1,"You know the answer man, you are programmed to...","4,27",ee84bjg
2,I've never been this sad in my life!,25,edcu99z
3,The economy is heavily controlled and subsidiz...,"4,27",edc32e2
4,He could have easily taken a real camera from ...,20,eepig6r


#### Data Preprocessing

In [11]:
df_train = df_train.drop([2], axis=1)
df_test = df_test.drop([2], axis=1)
df_dev = df_dev.drop([2], axis=1)

In [13]:
def makeclass(x):
  x = x.split(',')
  return x[0]

In [14]:
df_train[1] = df_train[1].map(makeclass)
df_test[1] = df_test[1].map(makeclass)
df_dev[1] = df_dev[1].map(makeclass)

In [15]:
df_train.columns = ['text', 'labels']
df_test.columns = ['text', 'labels']
df_dev.columns = ['text', 'labels']

In [16]:
df_test

,text,labels
0,I’m really sorry about your situation :( Altho...,25
1,It's wonderful because it's awful. At not with.,0
2,"Kings fan here, good luck to you guys! Will be...",13
3,"I didn't know that, thank you for teaching me ...",15
4,They got bored from haunting earth for thousan...,27
...,...,...
5422,Thanks. I was diagnosed with BP 1 after the ho...,15
5423,Well that makes sense.,4
5424,Daddy issues [NAME],27
5425,So glad I discovered that subreddit a couple m...,0


In [17]:
# Maps ids to emotion label

id2labels = {}
with open('/content/emotions.txt', 'r') as f:
  lines = f.readlines()
  lines = list(map(lambda x: x.strip("\n"), lines))

for i in range(28):
  id2labels[str(i)] = lines[i]

In [18]:
id2labels

{'0': 'admiration',
 '1': 'amusement',
 '10': 'disapproval',
 '11': 'disgust',
 '12': 'embarrassment',
 '13': 'excitement',
 '14': 'fear',
 '15': 'gratitude',
 '16': 'grief',
 '17': 'joy',
 '18': 'love',
 '19': 'nervousness',
 '2': 'anger',
 '20': 'optimism',
 '21': 'pride',
 '22': 'realization',
 '23': 'relief',
 '24': 'remorse',
 '25': 'sadness',
 '26': 'surprise',
 '27': 'neutral',
 '3': 'annoyance',
 '4': 'approval',
 '5': 'caring',
 '6': 'confusion',
 '7': 'curiosity',
 '8': 'desire',
 '9': 'disappointment'}

In [19]:
from sklearn.preprocessing import LabelBinarizer

In [20]:
onehot = LabelBinarizer()

In [21]:
train_onehot = onehot.fit_transform(df_train['labels'])
test_onehot = onehot.transform(df_test['labels'])
val_onehot = onehot.transform(df_dev['labels'])

In [20]:
df_train = pd.concat([df_train, pd.DataFrame(train_onehot)], axis=1)
df_test = pd.concat([df_test, pd.DataFrame(test_onehot)], axis=1)
df_dev = pd.concat([df_dev, pd.DataFrame(val_onehot)], axis=1)

In [21]:
df_train

,text,labels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,My favourite food is anything I didn't have to...,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,"Now if he does off himself, everyone will thin...",27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,WHY THE FUCK IS BAYLESS ISOING,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,To make her feel threatened,14,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Dirty Southern Wankers,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43406,Always thought that was funny but is it a refe...,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
43407,What are you talking about? Anything bad that ...,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
43408,"More like a baptism, with sexy results!",13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
# Save processed dataset for future purpose
df_train.to_csv("/content/train.csv")
df_test.to_csv("/content/test.csv")
df_dev.to_csv("/content/dev.csv")

#### Installing Dependencies

In [23]:
! pip install transformers

     |████████████████████████████████| 2.1MB 8.6MB/s 
     |████████████████████████████████| 901kB 52.8MB/s 
     |████████████████████████████████| 3.3MB 56.4MB/s 


In [24]:
! pip install datasets

     |████████████████████████████████| 204kB 8.0MB/s 
     |████████████████████████████████| 245kB 34.0MB/s 
     |████████████████████████████████| 112kB 37.9MB/s 


#### Model loading

In [25]:
import transformers
import datasets

In [26]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

#### Data preprocessing for model

In [30]:
train_encodings = tokenizer(df_train['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(df_dev['text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(df_test['text'].tolist(), truncation=True, padding=True)

In [31]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    df_train.iloc[:,2:]
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    df_dev.iloc[:,2:]
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    df_test.iloc[:,2:]
))

In [ ]:
train_dataset

<TensorSliceDataset shapes: ({input_ids: (316,), attention_mask: (316,)}, (28,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

#### Model Training

In [ ]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=28)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"]) # can also use any keras loss fn
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

In [ ]:
model.save_pretrained('./pretrained')

In [ ]:
tokenizer.save_pretrained("./pretrained")

('./pretrained/tokenizer_config.json',
 './pretrained/special_tokens_map.json',
 './pretrained/vocab.txt',
 './pretrained/added_tokens.json')

In [ ]:
import pickle
with open('./pretrained/labelbin.pickle', 'wb') as f:
    pickle.dump(onehot, f)

###Inference

In [ ]:
model1 = TFDistilBertForSequenceClassification.from_pretrained('./pretrained/')

Some layers from the model checkpoint at ./pretrained/ were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./pretrained/ and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(df_test.loc[0,'text'])
print(df_test.loc[0,'labels'])

I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!
25


In [ ]:
inputs = tokenizer("I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!", return_tensors="tf")

In [ ]:
output = model1(inputs)

In [ ]:
output = tf.nn.softmax(output['logits'])
output.numpy()

In [25]:
out = np.zeros(28, dtype='int')
ind = np.argmax(output.numpy())
out[ind] = 1
finallabel = onehot.inverse_transform(out.reshape([1,28]))
print("Predicted emotion: " + id2labels[finallabel[0]])

Predicted emotion: sadness


### Download Trained Model

In [ ]:
! zip -r /content/pretrained.zip /content/pretrained

  adding: content/pretrained/ (stored 0%)
  adding: content/pretrained/tokenizer_config.json (deflated 38%)
  adding: content/pretrained/tf_model.h5 (deflated 8%)
  adding: content/pretrained/labelbin.pickle (deflated 41%)
  adding: content/pretrained/config.json (deflated 65%)
  adding: content/pretrained/special_tokens_map.json (deflated 40%)
  adding: content/pretrained/vocab.txt (deflated 53%)
